### 1

Для каждого пользователя найти последние три взятые им произведения.
Для каждого такого произведения указать сколько всего раз его брали (за все
время).

In [ ]:
CREATE VIEW summary AS
  (SELECT id_user,
          id_произведение, Название, дата_взяли
   FROM Лог операций F
   LEFT JOIN
     (SELECT D.id AS id_экземпляр,
             id_издание,
             id_произведение, Название
      FROM Экземпляр AS D
      LEFT JOIN
        (SELECT A.id AS id_Издание,
                id_произведение, Название
         FROM Издание AS A
         LEFT JOIN Произведение AS B ON A.id_произведения = B.id) AS С ON D.id_издание = C.id_издание) AS E ON D.id_экземпляр = E.id_экземпляр
   ORDER BY id_user, дата_взяли DESC);

-- Для каждого пользователя найти последние три взятые им произведения.

CREATE VIEW last_books AS
  (SELECT G.id_user AS id_user,
          G.latest_date AS latest_date,
          1_book,
          2_book,
          3_book
   FROM
     (SELECT id_user,
             MAX(дата_взяли) AS latest_date
      FROM summary
      GROUP BY AS id_user) AS G
   JOIN
     (SELECT id_user, дата_взяли,
                          first_value(Название) OVER w AS 1_book,
                                                     nth_value(Название, 2) OVER w AS 2_book,
                                                                                 last_value(Название) OVER w AS 3_book
      FROM summary WINDOW w AS(PARTITION BY id_user
                               ORDER BY дата_взяли DESC RANGE ROWS BETWEEN CURRENT ROW AD 2 UNBONDED FOLLOWING)) AS H ON G.latest_date= H.дата_взяли
   AND G.id_user= H.id_user);


SELECT *
FROM last_books;

-- Для каждого такого произведения указать сколько всего раз его брали (за все время)

SELECT Название, COUNT(дата_взяли) AS times_ordered
FROM summary
WHERE Название IS IN
    (SELECT 1_book
     FROM last_books)
  OR Название IS IN
    (SELECT 3_book
     FROM last_books)
  OR Название IS IN
    (SELECT 2_book
     FROM last_books)
GROUP BY Название

### 2.1 Вывести детализацию по клиенту

a. ID клиента

b. Дата и сумма первой покупки

c. Дата и сумма повторной (следующей после первой) покупки

d. Дата последней покупки

e. Сумма покупок, совершенных в течение месяца после первой покупки

f. Время (кол-во дней) между первой и повторной покупкой

g. Среднее время (кол-во дней) между покупками

In [ ]:
SELECT L.ClientID AS ClientID,
       L.first_order_date AS first_order_date,
       second_day_date,
       second_day_order_sum,
       last_order_date,
       last_order_sum,
       first_month_sum,
       second_day_date - L.first_order_date AS time_bw_first_two_orders,
       avg_time_bw_orders
FROM
  (SELECT ClientID,
          order_date,
          day_sum,
          LEAD(order_date, 1) OVER w AS second_day_date,
                                   LEAD(day_sum, 1) OVER w AS second_day_order_sum,
                                                         LAST_VALUE(order_date) OVER w AS last_order_date,
                                                                                     LAST_VALUE(day_sum) OVER w AS last_order_sum,
                                                                                                              SUM(day_sum) OVER (PARTITION BY ClientID
                                                                                                                                 ORDER BY order_date ASC RANGE BETWEEN CURRENT ROW AND '1 month' FOLLOWING) AS first_month_sum
   FROM
     (SELECT ClientID,
             date(OperationTime) AS order_date,
             SUM(order_id) AS day_sum
      FROM tblClientBalanceOperation
      WHERE SignOfPayment=1
        AND ClientID NOT IN
          (SELECT ClientID
           FROM tblTestClients)
      GROUP BY ClientID,
               order_date) AS A WINDOW w AS (PARTITION BY ClientID
                                             ORDER BY order_date ASC RANGE BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)) AS R
JOIN
  (SELECT ClientID,
          MIN(order_date) AS first_order_date,
          AVG(second_order_date - order_date) AS avg_time_bw_orders
   FROM
     (SELECT ClientID,
             date(OperationTime) AS order_date,
             LEAD(order_date, 1) OVER (PARTITION BY ClientID
                                       ORDER BY order_date ASC) AS second_order_date
      FROM tblClientBalanceOperation
      WHERE SignOfPayment=1
        AND ClientID NOT IN
          (SELECT ClientID
           FROM tblTestClients) ) AS a
   GROUP BY ClientID) AS L ON R.ClientID = L.ClientID
AND R.order_date = L.first_order_date

### 2.2 Количество потерянных клиентов по месяцам

Потерянным клиентом считается тот, который совершал покупки в предыдущем
месяце, но не совершал в текущем. Какой % из этих потерянных клиентов
находятся в статусе Deleted?
Вывести по месяцам за текущий год

In [ ]:
SELECT AVG(st)
FROM
  (SELECT CASE
              WHEN Status = 'Deleted' THEN 1
              ELSE 0
          END AS st
   FROM
     (SELECT Status
      FROM
        (SELECT*,
               LAST_VALUE(month_num) OVER w AS prev_month_num
         FROM
           (SELECT ClientID,
                   max(OperationTime) AS months_last_order,
                   EXTRACT(MONTH
                           FROM OperationTime) AS month_num,
                   EXTRACT(YEAR
                           FROM OperationTime) AS year_num
            FROM tblTestClients
            WHERE EXTRACT(YEAR
                          FROM OperationTime) = 2020
              AND SignOfPayment=1
              AND ClientID NOT IN
                (SELECT ClientID
                 FROM tblTestClients)
            GROUP BY ClientID,
                     EXTRACT(MONTH
                             FROM OperationTime),
                     EXTRACT(YEAR
                             FROM OperationTime)) AS A WINDOW w AS(PARTITION BY ClientID
                                                                   ORDER BY month_num DESC RANGE BETWEEN CURRENT ROW AND 1 FOLLOWING)) AS L
      JOIN tblClients AS R ON L.ClientID = R.ClientID
      WHERE month_num - prev_month_num != 1) AS C) AS D

### 2.3 Покупки по неделям с момента регистрации

Сколько клиентов совершают первую покупку в первую неделю с регистрации на
сайте, сколько во вторую неделю, сколько на третьей неделе и сколько позже?
(вывести абсолютные значения и доли)

In [ ]:
SELECT cohorts,
       COUNT(cohorts) AS cohorts_size
FROM
  (SELECT CASE
              WHEN time_since_reg <= 7 THEN 1
              WHEN time_since_reg > 7
                   AND time_since_reg <= 15 THEN 2
              WHEN time_since_reg > 15
                   AND time_since_reg <= 22 THEN 3
              ELSE 4
          END AS cohorts
   FROM
     (SELECT L.ClientID AS ClientID,
             firs_order_date,
             reg_date,
             (firs_order_date - reg_date) AS time_since_reg
      FROM (
              (SELECT ClientID,
                      MIN(OperationTime) AS firs_order_date
               FROM tblClientBalanceOperation
               WHERE SignOfPayment=1
                 AND client_id NOT IN
                   (SELECT client_id
                    FROM tblTestClients)
               GROUP BY ClientID) AS L
            JOIN
              (SELECT ClientID,
                      MIN(OnlineTime) AS reg_date
               FROM tblOnlineSessions_mini
               GROUP BY ClientID) AS R ON L.ClientID = R.ClientID) AS B) AS C) AS D
GROUP BY cohorts